In [6]:
import cv2
import mediapipe as mp
import numpy as np
#mediapipe객체 생성
mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose
#비디오 캡쳐
video_path="blur2.mp4"
cap = cv2.VideoCapture(0)
cap_video=cv2.VideoCapture(video_path)

# 선택된 랜드마크 리스트
SELECTED_LANDMARKS = [
    'NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
    'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW',
    'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX',
    'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE',
    'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX'
]
#1~33까지 인덱스 중에서 위에서 고른 랜드마크의 고유 인덱스를 얻어서 S_L_I 변수에 저장
SELECTED_LANDMARK_INDICES = [mp_pose.PoseLandmark[landmark].value for landmark in SELECTED_LANDMARKS]
#관절의 각도를 계산해주는 함수
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    return np.degrees(angle)
#관절의 각도 계산 b관절을 기준으로 a관절과 c관절의 cos값을 계산. cos값이 -1~1까지 나오므로 코사인의 역함수 값을 취하면 0~180도가 나옴. cos(-1): 180 cos(0):90
#cos (0도) = 1   cos(90도) =0 cos (180도) = -1


def process_frame(frame, pose):
    frame.flags.writeable = False #프레임을 읽기 모드로 염
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # cv2를 활용해서 BGR을 RGB형식으로 변환(pose는 RGB형식 이미지를 사용)
    results = pose.process(frame) #pose 객체로 프레임을 처리해 관절의 좌표획득
    frame.flags.writeable = True #프레임을 쓰기 모드로 염
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) #cv2로 보여주기위해 이미지를 BGR형식으로 다시 바꿔줌

    if results.pose_landmarks: #랜드마크가 있다면
        landmarks = results.pose_landmarks.landmark #감지된 랜드마크를 landmarks 변수에 할당
        connections_to_draw = [
            connection for connection in mp_pose.POSE_CONNECTIONS # MediaPipe에서 정의한 포즈 랜드마크 간의 기본 연결 목록을 돌면서 connection로 읽어들임
            if connection[0] in SELECTED_LANDMARK_INDICES and connection[1] in SELECTED_LANDMARK_INDICES
        ] #랜드 마크 고유 인덱스를 돌면서 랜드마크 연결목록이 둘다 랜드마크 인덱스에 있다면 connections_to_draw 리스트에 추가
        
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, connections_to_draw,
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )#이미지 프레임 위에 landmarks를 그림
        
# results.pose_landmarks:랜드마크 객체
# connections_to_draw:선택된 랜드마크 간의 연결 리스트
# mp_drawing.DrawingSpec: 연결할때 스타일 지정 첫번째는 랜드마크 점의 스타일 두번째는 선의 스타일



    
    return frame, results
#관절의 좌표값을 얻어서 angles리스트에 담아 cal angles함수로 보낸 후 나온 각도 결과값을 angles에 순서대로 저장
def get_angles(landmarks):
    angles = []
    
    # 오른쪽 팔꿈치 각도
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
    angles.append(angle)

    # 왼쪽 팔꿈치 각도
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
    angles.append(angle)
    
    # 오른쪽 무릎 각도
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    angle = calculate_angle(right_hip, right_knee, right_ankle)
    angles.append(angle)  # 수정: 튜플 대신 숫자만 추가

    # 왼쪽 무릎 각도
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    angle = calculate_angle(left_hip, left_knee, left_ankle)
    angles.append(angle)  # 수정: 튜플 대신 숫자만 추가


    # 오른쪽 발목 각도
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
    angle = calculate_angle(right_knee, right_ankle, right_foot_index)
    angles.append(angle)

    # 왼쪽 발목 각도
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
    angle = calculate_angle(left_knee, left_ankle, left_foot_index)
    angles.append(angle)

    # 오른쪽 골반 각도
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    angle = calculate_angle(right_shoulder, right_hip, right_knee)
    angles.append(angle)

    # 왼쪽 골반 각도
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    angle = calculate_angle(left_shoulder, left_hip, left_knee)
    angles.append(angle)

    return angles

# 메인 루프 부분
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose: 
    #min_detection_confidence와 min_tracking_confidence는 각각 최소 감지 신뢰도와 최소 추적 신뢰도를 설정하고 Pose객체를 초기화
    while cap.isOpened() and cap_video.isOpened():#웹캠의 cap이 열려있는동안 반복실행
        ret, frame = cap.read()#cap의 프레임이 제대로 들어갔는지에 대한 불리언값(ret)과 그 프레임의 이미지(frame)를 저장
        ret_video, frame_video = cap_video.read() #비디오에 대해서도 실행

        if not ret or not ret_video: #ret이나 ret_video에 False가 들어가면 프린트 즉 영상이 끝나거나 웹캠이 꺼졌다면 break
            print("Can't receive frame (stream end?). Exiting ...")
            break

        frame, results = process_frame(frame, pose) #이미지에 대해 process_frame를 적용해서 frame(랜드마크가 그려진 이미지)과 results(관절 좌표)를 얻음
        frame_video, results_video = process_frame(frame_video, pose)#비디오이미지에 대해 process_frame를 적용해서 frame(랜드마크가 그려진 이미지)과 results(관절 좌표)를 얻음

        if results.pose_landmarks and results_video.pose_landmarks: #두 이미지의 results에 랜드마크가 있으면
            angles_webcam = get_angles(results.pose_landmarks.landmark) #웹캠의 각도를 얻음
            angles_video = get_angles(results_video.pose_landmarks.landmark)#비디오의 각도를 얻음
            
            angle_differences = np.array(angles_webcam) - np.array(angles_video) #웹캠과 비디오의 차이를 얻음
            
            # 각도 차이를 화면에 표시
            joint_names = ["Right Elbow", "Left Elbow", "Right Knee", "Left Knee", 
               "Right Ankle", "Left Ankle", "Right Hip", "Left Hip"]
            for i, (name, diff) in enumerate(zip(joint_names, angle_differences)):
                cv2.putText(frame, f"{name}: {diff:.2f}", 
                (10, 30 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            # 각도 차이 배열 출력
            print(f"Angle differences: {angle_differences}")

        # 이미지 크기 맞추기
        height = min(frame.shape[0], frame_video.shape[0])
        width = min(frame.shape[1], frame_video.shape[1])

        frame_resized = cv2.resize(frame, (width, height))
        frame_video_resized = cv2.resize(frame_video, (width, height))

        # 두 영상을 나란히 표시
        combined_frame = np.hstack((frame_resized, frame_video_resized))
        cv2.imshow('Webcam and Video Comparison', combined_frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cap_video.release()
cv2.destroyAllWindows()
#같은 영상으로 관절 각도의 차이를 봤을때 이론상 0이 나와야 맞지만 개개인의 키와 특성을 고려했을때 아래 수치는 오차값이 작은 수준이므로 위의 각도 계산식으로 운동법 피드백을 위한 참고 자료로 사용가능하다고 판단.
#openpose와 비교했을 때 mediapipe의 실시간 추적이 더 빠르기때문에 실시간 웹캠과 비교하기에 더 적합하다고 판단하여 mediapipe사용
#openpose는 bottom-up방식으로 인물이 늘어나도 비슷한 계산시간을 소요하지만
#mediapipe는 top-down방식으로 인물 객체를 먼저 찾고 관절로 내려가기 떄문에 인물수에 따라 계산 시간이 선형적으로 증가

#angle differences = 웹캠비디오의 해당 관절의 각도 - 운동비디오의 해당관절의 각도
#angle differences 값이 양수이면 웹캠의 해당 관절의 각도가 운동비디오의 각도보다 크다 -> 해당 관절을 더 굽혀야 운동비디오의 각도와 같아짐
#angle differences 값이 음수이면 웹캠의 해당 관절의 각도가 운동비디오의 각도보다 작다 -> 해당 관절을 더 펴야 운동 비디오의 각도와 같아짐

Angle differences: [  29.76361659    4.67614541  -90.92231316   14.46415947  -52.35627363
   26.55253851 -156.55032926 -170.49585037]
Angle differences: [  18.59032225  -41.03726284   56.30009084   25.13398202  -40.09677131
 -119.26127917 -161.64161006 -168.40729716]
Angle differences: [  83.85556028 -140.50286428   68.74753471   31.82092643   37.01038006
  -65.03208418 -153.82372255 -169.98312106]
Angle differences: [  55.52451385   68.96038362  -21.46224007  -21.89630115    3.33561193
  -29.88010187 -148.52452611 -165.0510582 ]
Angle differences: [33.86649487 39.8616692  14.69534644 23.22917279 -1.88172902 76.46321132
  0.45289037 -2.38328782]
Angle differences: [  46.47060402  -58.23150654  -77.90340359 -157.2624091    -0.28465548
 -117.9060275  -165.17538483 -172.77970425]
Angle differences: [-102.85733901  -96.35735155  -28.78158024  -88.50549075  -42.40871648
  -59.35016677  -18.49392294  -10.30836751]
Angle differences: [  -4.25577816  -67.01992023  -20.40565274  -38.16907883  -